In [ ]:
import numpy as np
from TMDP import TMDP
from RiverSwim import River

from algorithms import *
from model_functions import *

import matplotlib.pyplot as plt



#np.set_printoptions(precision=4)
import math
from utils import *

nS = 350
nA = 2
seed = get_current_seed()
seed = 47146882006333120128184314011326542902
gamma = .9
mu = np.ones(nS) * 1/nS
river = River(nS, mu, small=5, large=1000, seed=seed)
tau = 1.
xi = np.ones(nS) * 1/nS
tmdp = TMDP(river, xi, tau=tau, gamma=gamma, seed=seed)
mdp = TMDP(river, xi, tau=0., gamma=gamma, seed=seed)
tmdp_1 = TMDP(river, xi, tau=.5, gamma=gamma, seed=seed)


In [ ]:
q_c = np.zeros((nS, nA))
tmdp.reset()
cur_res = curriculum_Q_learning(tmdp, q_c, curriculum=[0.], alpha = 0.08, eps = 0.2, episodes=1500000, 
                        batch_nS=1, lam=0)


In [ ]:
res = bellman_optimal_q(mdp.env.P_mat, mdp.env.reward, mdp.gamma)
Q = res["Q"]
V = compute_V_from_Q(Q, get_policy(Q))
#print(Q)

delta_pol = get_policy(Q)-get_policy(q_c)
print(delta_pol)


V_curr = compute_V_from_Q(Q, get_policy(q_c))


j_curr = compute_expected_j(V_curr, tmdp.env.mu)
j_opt = compute_expected_j(V, tmdp.env.mu)
print("optimal performance: ",j_opt)
print("curriculum performance: ", j_curr)

In [ ]:
import matplotlib.pyplot as plt
reward_records = cur_res["reward_records"]
# Generate recent 50 interval average
average_reward = []
for idx in range(len(reward_records)):
    avg_list = np.empty(shape=(1,), dtype=int)
    if idx < 500:
        avg_list = reward_records[:idx+1]
    else:
        avg_list = reward_records[idx-499:idx+1]
    average_reward.append(np.average(avg_list))
# Plot
#plt.plot(reward_records)
plt.plot(average_reward)

In [ ]:
import matplotlib.pyplot as plt
exp_performances = cur_res["exp_performance"]
# Generate recent 50 interval average
avg_performances = []
for idx in range(len(exp_performances)):
    avg_list = np.empty(shape=(1,), dtype=int)
    if idx < 500:
        avg_list = exp_performances[:idx+1]
    else:
        avg_list = exp_performances[idx-499:idx+1]
    avg_performances.append(np.average(avg_list))
# Plot
plt.plot(exp_performances)
#plt.plot(avg_performances)